In [23]:
import spacy
import os
import csv
import sys


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

nlp = spacy.load('es_core_news_sm')

In [24]:
#cluster victor
def load_stopwords(string):
    """
    Lee archivos .txt con stopwords. Deben estar separadas por salto de linea
    :param string:
    :return:
    """
    print("Reading Stopwords")
    # Si el archivo existe
    with open(string, 'r', encoding="utf8") as words:
        sw = words.read().split('\n')
    return sw
stop_words = load_stopwords('stopwords_es_test.txt')


Reading Stopwords


In [25]:
#https://stackoverflow.com/questions/15063936/csv-error-field-larger-than-field-limit-131072
data = []
president='bachelet'
with open('data/csv/union.csv') as csvfile:
    maxInt = sys.maxsize
    while True:
        # decrease the maxInt value by factor 10
        # as long as the OverflowError occurs.

        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt / 10)
    readCSV = csv.DictReader(csvfile)
    for row in readCSV:
        if row['president']== president:
            content=str(row['content']).lower()
            data.append(content)
print(data[0])


amigas y amigos: ésta es mi quinta intervención en esta cena de la energía partió la primera en mi último año de gobierno anterior y será mi última intervención en esta cena de energía como presidenta. en este período han sido algo más de tres años y sin embargo hemos recorrido un enorme camino. la primera vez que estuve aquí teníamos mucho por hacer había tareas urgentes había incertidumbre sobre qué ocurriría si no nos poníamos de inmediato a la acción. teníamos eso sí una hoja de ruta de largo plazo y teníamos la voluntad conjunta de echarla a andar cuanto antes. teníamos necesidad de construir confianza y eso es lo que hemos hecho. “nos medirán por nuestras acciones” dije entonces y estoy feliz de que así sea. porque es mucho lo que ha ocurrido desde esa primera cena –segunda en este caso primera en este gobierno– desde ese diagnóstico. y juntos hemos sido protagonistas de lo ocurrido. y hoy día aquí veo rostros conocidos; nos hemos encontrado en más de una ocasión en torno a algun

In [26]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)




In [25]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)



['amigas_amigos', 'esta', 'es', 'mi', 'quinta', 'intervencion', 'en', 'esta', 'cena', 'de', 'la', 'energia', 'partio', 'la', 'primera', 'en', 'mi', 'ultimo', 'ano', 'de', 'gobierno', 'anterior', 'sera', 'mi', 'ultima', 'intervencion', 'en', 'esta', 'cena', 'de', 'energia', 'como', 'presidenta', 'en', 'este', 'periodo', 'han', 'sido', 'algo', 'mas', 'de', 'tres', 'anos', 'sin_embargo', 'hemos', 'recorrido', 'un', 'enorme', 'camino', 'la', 'primera', 'vez', 'que', 'estuve', 'aqui', 'teniamos', 'mucho', 'por', 'hacer', 'habia', 'tareas', 'urgentes', 'habia', 'incertidumbre', 'sobre', 'que', 'ocurriria', 'si', 'no', 'nos', 'poniamos', 'de', 'inmediato', 'la', 'accion', 'teniamos', 'eso', 'si', 'una', 'hoja', 'de', 'ruta', 'de', 'largo_plazo', 'teniamos', 'la', 'voluntad', 'conjunta', 'de', 'echarla', 'andar', 'cuanto', 'antes', 'teniamos', 'necesidad', 'de', 'construir', 'confianza', 'eso', 'es', 'lo', 'que', 'hemos', 'hecho', 'nos', 'mediran', 'por', 'nuestras', 'acciones', 'dije', 'enton

In [26]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

print(id2word[0])
#corpora.MmCorpus.serialize('newsgroups.mm', corpus)
#id2word.save('newsgroups.dict')
print([[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]])



[['amigas_amigos', 'quinto', 'intervencion', 'cenar', 'energia', 'partio', 'intervencion', 'cenar', 'energia', 'presidente', 'periodo', 'recorrer', 'enorme', 'teniamos', 'tarea', 'urgente', 'incertidumbre', 'ocurriria', 'poniamos', 'inmediato', 'accion', 'teniamos', 'hoja', 'ruta', 'plazo', 'teniamos', 'voluntad', 'conjunto', 'echarla', 'andar', 'teniamos', 'necesidad', 'construir', 'confianza', 'mediran', 'accionar', 'feliz', 'ocurrir', 'cenar', 'casar', 'diagnosticar', 'protagonista', 'ocurrir', 'ver', 'rostro', 'conocido', 'encontrar', 'ocasion', 'tornar', 'hito', 'agenda', 'energia', 'referente', 'regional', 'casar', 'exito', 'transicion_energetica', 'evidenciar', 'parecio', 'desmedir', 'realista', 'simplemente', 'secundario', 'salar', 'contribuir', 'sector', 'convertir', 'dinamico', 'economia', 'mero', 'preocupacion', 'economica', 'gremial', 'politica', 'industriar', 'ver', 'enorme', 'convergencia', 'voluntad', 'compromiso', 'vision', 'tranquilidad', 'marchar', 'caida', 'precio', 

[[(0, 2), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 4), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 3), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 2), (35, 1), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 3), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 2), (60, 3), (61, 1), (62, 2), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 2), (71, 3), (72, 3), (73, 7), (74, 1), (75, 1), (76, 1), (77, 6), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1), (90, 3), (91, 1), (92, 1), (93, 1), (94, 4), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 2), (101, 1), (102, 1), (103, 2), (104, 1), (105, 1), (106, 2), (107, 1), (108, 1), (109, 1), (110, 1)

In [27]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
#lda_model.save('newsgroups_50_lda.model')
print(lda_model.print_topics())
doc_lda = lda_model[corpus]
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.



[(0, '0.020*"mujer" + 0.007*"grotius" + 0.007*"japon" + 0.006*"rights" + 0.006*"igualdad" + 0.005*"humanitario" + 0.005*"derecho" + 0.005*"world" + 0.005*"paz" + 0.005*"tpp"'), (1, '0.012*"country" + 0.007*"global" + 0.006*"development" + 0.005*"caricom" + 0.005*"challenge" + 0.004*"new" + 0.004*"economic" + 0.004*"social" + 0.004*"international" + 0.004*"region"'), (2, '0.019*"publicar" + 0.018*"derecho" + 0.016*"ciudadano" + 0.014*"democracia" + 0.011*"institución" + 0.011*"politica" + 0.009*"ciudadania" + 0.009*"justicia" + 0.009*"delito" + 0.009*"reformar"'), (3, '0.019*"mujer" + 0.015*"salud" + 0.012*"familia" + 0.010*"calidad" + 0.009*"programar" + 0.008*"ninos" + 0.008*"trabajar" + 0.008*"mejorar" + 0.007*"vida" + 0.006*"jovenes"'), (4, '0.011*"region" + 0.010*"trabajar" + 0.009*"obrar" + 0.007*"mejorar" + 0.006*"plan" + 0.006*"vivienda" + 0.006*"contar" + 0.006*"permitir" + 0.006*"casar" + 0.005*"familia"'), (5, '0.009*"mujer" + 0.007*"sociedad" + 0.007*"historia" + 0.006*"vida

In [28]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.5079235057390654


In [ ]:
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)


In [30]:
pyLDAvis.display(vis)##arreglar